---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Understanding and Predicting Property Maintenance Fines
This project is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

[Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this project, I created a model to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). 
___

Two data files were used for training and validating the models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

The predictions will be given as the probability that the corresponding blight ticket will be paid on time. This model will return a series of length 61001 with the data being the probability that each corresponding ticket from test.csv will be paid, and the index being the ticket_id.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

In [49]:
import pandas as pd
import numpy as np

def blight_model():
#     pd.set_option('display.max_rows', None)
    from sklearn.preprocessing import LabelEncoder
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score
    
    train = pd.read_csv(r"C:\Users\blord\Documents\Code Projects\Coursera\Applied Machine Learning in Python\Files\home\jovyan\work\readonly\train.csv", encoding = "ISO-8859-1")
    test = pd.read_csv(r"C:\Users\blord\Documents\Code Projects\Coursera\Applied Machine Learning in Python\Files\home\jovyan\work\readonly\test.csv")
    addresses = pd.read_csv(r"C:\Users\blord\Documents\Code Projects\Coursera\Applied Machine Learning in Python\Files\home\jovyan\work\readonly\addresses.csv")
    latlons = pd.read_csv(r"C:\Users\blord\Documents\Code Projects\Coursera\Applied Machine Learning in Python\Files\home\jovyan\work\readonly\latlons.csv")

#     train = train.set_index('ticket_id')
    train = train[~(np.isnan(train['compliance']))]
    train = pd.merge(train, pd.merge(addresses, latlons, on = 'address'), on = 'ticket_id')
    train = train[train['country'] == 'USA']
    train = train.drop(['agency_name', 'inspector_name', 'violator_name', 'non_us_str_code',  
                'grafitti_status', 'ticket_issued_date', 'hearing_date','payment_amount','payment_date',
               'payment_status', 'balance_due','collection_status','compliance_detail','violation_street_number',
                'violation_street_name','violation_zip_code','mailing_address_str_number','mailing_address_str_name','city','state','zip_code','non_us_str_code',
                'country','lat', 'lon'],axis =1)
    le = LabelEncoder()
    train['address'] = le.fit_transform(train['address'])
    train['disposition'] = le.fit_transform(train['disposition'])
    train['violation_code'] = le.fit_transform(train['violation_code'])
    train['violation_description'] = le.fit_transform(train['violation_description'])
#     train = train[~(np.isnan(train['lat']))]
#     train = train[~(np.isnan(train['lon']))]
#     train = train[~(np.isnan(train['address']))]
    train = train.set_index('ticket_id')
    y = train['compliance']
    X = train.drop(['compliance',], axis = 1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
    clf = GradientBoostingClassifier().fit(X_train, y_train)
    
    test = pd.merge(test, pd.merge(addresses, latlons, on = 'address'), on = 'ticket_id')
    test = test[test['country'] == 'USA']
    test = test.drop(['agency_name', 'inspector_name', 'violator_name', 'non_us_str_code',  
                'grafitti_status', 'ticket_issued_date', 'hearing_date','violation_street_number',
                'violation_street_name','violation_zip_code','mailing_address_str_number','mailing_address_str_name','city','state','zip_code','non_us_str_code',
                'country','lat','lon'],axis =1)
    test['address'] = le.fit_transform(test['address'])
    test['disposition'] = le.fit_transform(test['disposition'])
    test['violation_code'] = le.fit_transform(test['violation_code'])
    test['violation_description'] = le.fit_transform(test['violation_description'])
#     test = test[~(np.isnan(test['lat']))]
#     test = test[~(np.isnan(test['lon']))]
#     test = test[~(np.isnan(test['address']))]
    test = test.set_index('ticket_id')
    prediction_array = clf.predict_proba(test)[:,1]
    compliance = pd.Series(prediction_array,index=test.index)
    return compliance

In [50]:
blight_model()

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DtypeWarning: Columns (11,12,31) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


ticket_id
284932    0.274379
285362    0.060206
285361    0.152820
285338    0.158625
285346    0.201300
            ...   
376496    0.061396
376497    0.061396
376499    0.212309
376500    0.212309
369851    0.857584
Length: 61001, dtype: float64